# 5. Topic Labelling
In this notebook we will be infering a label for each one of the topics obtained in the Topic Modelling notebook. The main process will be divided as follows:
* We will begin by linking to [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) each one of the terms that belong to each topic. (_Entity linking_)
* After the entity linking step, we will generate a Graph with the neighbour nodes of each starting term by traversing their Wikidata information based on a set of properties to expand.
* Once a graph is built for each topic, we will remove isolated subgraphs from them to obtain the main connected subgraph of each one.
* We will apply a set of centrality algorithms to obtain the entity from Wikidata which best represents the topic subgraph. This node will be used as a label for the topic.
* Finally, the topic labels will be added to the final model, which will be serialized for further use in the following notebooks.

## Setup
As always, we will begin by loading a set of constants and initializing the logging system. Since we will be using Bokeh in this notebook, we will configure it to output the results in the Jupyter notebook:

In [1]:
%run __init__.py

In [2]:
from bokeh.io import output_notebook

output_notebook()

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

## Entity linking

### Using the entity linking class
An entity linking class has been defined in the _entity_linking.py_ module of the _src_ directory. This class will link the given words to their Wikidata entity by using the [wbsearchentities](https://www.wikidata.org/w/api.php?action=help&modules=wbsearchentities) modules from the MediaWiki API:

In [3]:
from src.entity_linking import WikidataEntityLinker

entity_linker = WikidataEntityLinker()
res = entity_linker.link_entity('agroforestry')
res

('agroforestry', 'http://www.wikidata.org/entity/Q397350')

### Linking each topic's term to Wikidata
In the following cells we are going to load the lda model trained on the Agriculture dataset, obtain the term distribution of each topic, and link each term to Wikidata. We will start by loading both the LDA pipeline and the document term matrix with the term frequency: 

In [46]:
from src.utils import load_object

lda_agriculture_pipe_filename = "agriculture_lda_model.pkl"
dtm_tf_filename = "agriculture_dtm_tf.pkl"

lda_pipe = load_object(os.path.join(NOTEBOOK_3_RESULTS_DIR, lda_agriculture_pipe_filename))
dtm_tf = load_object(os.path.join(NOTEBOOK_3_RESULTS_DIR, dtm_tf_filename))

In order to obtain the list of terms for each topic, we are going to make use of the _get\_topic\_terms\_by\_relevance_ function to obtain a list of more relevant terms for each topic (see [Sievert & Shirley](https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf) for more information).

In [5]:
from src.utils import get_topic_terms_by_relevance

def link_topic_terms(entity_linker, model, vectorizer,
                     dtm_tf, n_top_words, lambda_=0.6):
    res = []
    topic_terms = get_topic_terms_by_relevance(model, vectorizer, dtm_tf,
                                               n_top_words, lambda_)
    return [[entity_linker.link_entity(entity) for entity in topic]
            for topic in topic_terms]


Finally, we can make used of the function defined above to link each term to Wikidata. The output of the following cell will be a 2D array, with the first dimension corresponding to each topic, and the second one consisting on tuples containing the pair ('term', 'wikidata_uri') for every term of the topic:

In [6]:
linked_terms = link_topic_terms(entity_linker, lda_pipe.named_steps['model'],
                                lda_pipe.named_steps['vectorizer'], dtm_tf, 
                                n_top_words=10, lambda_=0.8)
linked_terms

[[('food', 'http://www.wikidata.org/entity/Q2095'),
  ('system', 'http://www.wikidata.org/entity/Q58778'),
  ('habitat', 'http://www.wikidata.org/entity/Q52105'),
  ('grassland', 'http://www.wikidata.org/entity/Q1006733'),
  ('production', 'http://www.wikidata.org/entity/Q739302'),
  ('agricultural', 'http://www.wikidata.org/entity/Q5356428'),
  ('land', 'http://www.wikidata.org/entity/Q11081619'),
  ('bird', 'http://www.wikidata.org/entity/Q5113'),
  ('change', 'http://www.wikidata.org/entity/Q1150070'),
  ('security', 'http://www.wikidata.org/entity/Q2526135')],
 [('plant', 'http://www.wikidata.org/entity/Q756'),
  ('expression', 'http://www.wikidata.org/entity/Q11024'),
  ('gene', 'http://www.wikidata.org/entity/Q7187'),
  ('adaptation', 'http://www.wikidata.org/entity/Q3331189'),
  ('root', 'http://www.wikidata.org/entity/Q111029'),
  ('shoot', 'http://www.wikidata.org/entity/Q220869'),
  ('site', 'http://www.wikidata.org/entity/Q35127'),
  ('diversification', 'http://www.wikidata.

## Obtaining each topic's graphs
In this phase we are going to explore the neighbourhood of each term linked before, to obtain a graph with their related terms from Wikidata. Each set of terms obtained before will be the seed concepts used to obtain the final graph, and a set of properties from Wikidata will be explored recursively to expand the final graph. 

For more information about the implementation of the graph building process, the class used can be accessed at the _graph.py_ module in the source directory.

In the following cell we will be configuring the graph builder to build a graph with a maxium depth of two from every seed node. Higher depth values might cause the resulting topic labels to be very general, while with a smaller value we have the risk of not obtaining a connection between the seed nodes:

In [7]:
from src.graph import WikidataGraphBuilder

graph_builder = WikidataGraphBuilder(max_hops=2)
topic_graphs = [graph_builder.build_graph(topic) for topic in linked_terms]

Now that we have obtained the neighbourhood graph of each topic, we are going to plot the results using bokeh. Each node will have a different color depending on their depth with respect to the seed nodes, which will be painted in blue. This will allow us to perform an initial exploration of these graphs:

In [8]:
from bokeh.io import show
from bokeh.layouts import gridplot

from src.graph import build_graph_plot


plots = [build_graph_plot(g, f"Topic {idx}") 
         for idx, g in enumerate(topic_graphs)]
grid = gridplot(plots, ncols=2)
show(grid)

An optimum result would be to have every seed term connected in the final graph. However, theere will be some subgraphs which are isolated from the main ones. In the following section we will be solving this issue.

## Getting the main connected subgraph
As we have described before, some of the topic graphs that we have obtained are not fully connected. Small subgraphs which are isolated from the main subgraph will be considered as noise, and removed before the following computations.

In the following cells, we are going to retrieve the largest connected subgraph from each topic's graph, and plot the results to anaylise them:

In [9]:
from src.graph import get_largest_connected_subgraph

connected_topic_subgraphs = [get_largest_connected_subgraph(g) 
                             for g in topic_graphs]

In [10]:
plots = [build_graph_plot(g, f"Largest Connected subgraph for topic {idx}") 
         for idx, g in enumerate(connected_topic_subgraphs)]
grid = gridplot(plots, ncols=2)
show(grid)

In this section we are aiming to see big graphs with the most amount of seed nodes possible. Graphs with few seed nodes from the original term distribution will tend to be less representative of the original topic.

## Obtaining the main component of each topic
Now that we have the final subgraph for each topic, we will be applying several centrality measures to obtain the node that best represents the topic. In the following cell we have defined an auxiliary function that receives a list of algorithm and returns the results of applying them to obtain the best _n_ entities that represent each topic:

In [11]:
import networkx.algorithms as nxa

from src.graph import get_centrality_algorithm_results

def try_centrality_algorithms(topic_subgraphs, algorithms, stop_uris, top_n=4):
    markdown = ""
    for (algorithm, name) in algorithms:
        print(f'Algorithm: {name}')
        results = [get_centrality_algorithm_results(g, algorithm, stop_uris, top_n)
                   for g in topic_subgraphs]
        for idx, result in enumerate(results):
            print(f"Topic {idx}:", result)
            print()
        print()

        
algorithms = [
    (nxa.centrality.information_centrality, "Information centrality"),
    (nxa.centrality.eigenvector_centrality_numpy, "Eigenvector centrality"),
    (nxa.centrality.closeness_centrality, "Closeness centrality"),
    (nxa.centrality.betweenness_centrality, "Betweenness centrality"),
    (nxa.centrality.communicability_betweenness_centrality, "Communicability betweenness centrality")
]

try_centrality_algorithms(connected_topic_subgraphs,
               algorithms,
               ['Q4167836', 'Q11862829'])

Algorithm: Information centrality
Topic 0: [('action', 0.003380982345030935), ('security engineering', 0.003212948613959991), ('activity', 0.0031184946504963797), ('eating', 0.003074788823943842)]

Topic 1: [('interaction science', 0.002078526615343087), ('communication studies', 0.0019802059353208404), ('communication medium', 0.0019580563743009844), ('botany', 0.00195255616921488)]

Topic 2: [('agriculture', 0.005156184827487581), ('economic sector', 0.004959247934727366), ('economic branch', 0.004777418468751681), ('cybernetics', 0.004530887177086316)]

Topic 3: [('agriculture', 0.00256209016173155), ('cultivated plant', 0.0024458065821052945), ('food ingredient', 0.002367814856148407), ('crop', 0.0023499439466720387)]

Topic 4: [('biological process', 0.0021336430604133494), ('group or class of chemical substances', 0.0019653123198158677), ('first-order metaclass', 0.0019392495946736255), ('biopolymer', 0.0019193038114239502)]

Topic 5: [('class', 0.006044313418359505), ('group or 

## Add labels to LDA model
Finally, we will be saving the best results to our LDA model that has been trained previously. Now, when we load the model again, after a topic has been inferred for a given text we will also be able to return a representative label for the topic, which will be also linked to Wikidata:

In [60]:
from sklearn.base import TransformerMixin, BaseEstimator

class LabelledTopicModel(BaseEstimator, TransformerMixin):
    def __init__(self, topic_model, labels, num_topics_returned=3):
        assert topic_model.n_components == len(labels)
        self.topic_model = topic_model
        self.labels = labels
        self.num_topics_returned = num_topics_returned
    
    def fit(self, X, y=None):
        self.topic_model.fit(X)
    
    def transform(self, X, *args, **kwargs):
        topic_distr = self.topic_model.transform(X)
        best_topics_idx = [np.argsort(text_topics)[::-1] 
                           for text_topics in topic_distr]
        return [[(self.labels[topic_idx], topic_distr[text_idx][topic_idx]) 
                 for topic_idx in text_topics[:self.num_topics_returned]]
                 for text_idx, text_topics in enumerate(best_topics_idx)]


In [53]:
final_results = [get_centrality_algorithm_results(g,
                                                 nxa.centrality.information_centrality,
                                                ['Q4167836', 'Q11862829'], top_n=1)
                 for g in connected_topic_subgraphs]

final_results_labels = [topic[0] for result in final_results for topic in result]
lda_model = lda_pipe.named_steps['model']

In [61]:
labelled_topic_model = LabelledTopicModel(lda_model, final_results_labels)

lda_pipe.steps.pop()
lda_pipe.steps.append(('model', labelled_topic_model))

lda_pipe.transform([publications[-5]])

1it [00:00,  1.21it/s]


[[('biological process', 0.9470693118954655),
  ('organism', 0.033808853993507214),
  ('biological process', 0.018859669490589297)]]

In [18]:
from src.utils import save_object

save_object(lda_pipe, os.path.join(NOTEBOOK_5_RESULTS_DIR, 'lda_pipe_with_labels.pkl'))